This notebook is for downloading historical weather data from: https://climate.weather.gc.ca/

In [ ]:
import pandas as pd
import requests
from io import StringIO

Create a list of weather stations, and associated daily range date values, near lat/long coordinates.

In [ ]:
url = "https://climate.weather.gc.ca/historical_data/search_historic_data_stations_e.html"

params = {
    "searchType":"stnProx",
    "timeframe":2,
    "txtRadius":25,
    "optProxType":"decimal",
    "txtLatDecDeg":53.9169,
    "txtLongDecDeg":-122.7494,
    "optLimit":"yearRange",
    "StartYear":1840,
    "EndYear":2021,
    "Year":2021,
    "Month":6,
    "Day":26,
    "selRowPerPage":25,
    "selCity": "",
    "selPark": "",
    "txtCentralLatDeg": "",
    "txtCentralLatMin": "",
    "txtCentralLatSec": "",
    "txtCentralLongDeg": "",
    "txtCentralLongMin": "",
    "txtCentralLongSec": "",
}

download = requests.get(url, params=params)

sta_id_split = str(download.content).split('name="StationID" value="')
dlyRange_split = str(download.content).split('name="dlyRange" value="')

sta_ids = list(set([i.split('"')[0] for i in sta_id_split[1:]]))
dly_ranges = [i.split('"')[0] for i in dlyRange_split[1:]][:len(sta_ids)]
print(f"Found {len(sta_ids)} stations")

Download the weather data. If `use_dly_range` is set to `True`, the daily range dates collected above will be used. If not, all data between 1900 and 2021 will be downloaded at each weather station.

In [ ]:
url = "https://climate.weather.gc.ca/climate_data/bulk_data_e.html"
params = {
    "format": "csv",
    "timeframe": 2,
    "submit": "Download+Data"
}
dfs = []

use_dly_range = False

for j, sta_name in enumerate(sta_ids):
    if use_dly_range:
        split_dly_range = dly_ranges[j].split("|")
        start_year = int(split_dly_range[0][:4])
        end_year = int(split_dly_range[1][:4])
        print(start_year, end_year)
    else:
        start_year = 1900
        end_year = 2021
        
    for i in range(start_year, end_year+1):
        params["Year"] = i
        params["stationID"] = sta_name
        download = requests.get(url, params=params)
        
        if download.status_code == 200:
            decoded_content = StringIO(download.content.decode('utf-8'))
            dfs.append(pd.read_csv(decoded_content))
        else:
            print(download.status_code, i, sta_name)
    

Combine the results into a single DataFrame, and print the first few rows.

In [ ]:
df = pd.concat(dfs, axis=0, ignore_index=True)
df.head()

In [ ]:
df.shape

Notice above there are many rows where `Max Temp (°C)` and other attributes are NaN. Select the more useful data into a new DataFrame.

In [ ]:
dropped_na = df[df['Max Temp (°C)'].notna()]
dropped_na.reset_index(inplace=True)
dropped_na.shape

Save the combined, cleaned data to `csv`.

In [ ]:
dropped_na.to_csv("pg_historical_weather.csv")

Optionally, load the data back from the csv.

In [ ]:
dropped_na = pd.read_csv("pg_historical_weather.csv")

You can find record temperatures like below. For example, we find the maximum temperature recorded in June of any year before 2021.

In [ ]:
dropped_na.iloc[dropped_na[(dropped_na["Month"]==6) & (dropped_na["Year"]<2021)]['Max Temp (°C)'].idxmax()]

We can use HoloViews for nice interactive charts.

In [ ]:
import holoviews as hv
from bokeh.models import HoverTool
hv.extension('bokeh')

In [ ]:
hot_june = dropped_na[(dropped_na["Max Temp (°C)"]>30) & (dropped_na["Month"]==6)]
dt = pd.to_datetime(hot_june["Date/Time"])
hot_june["dt"] = dt
hot_june["max"] = hot_june["Max Temp (°C)"]
hot_june["sta"] = hot_june["Station Name"]

tooltips = [
    ('dt', '@dt{%F}'),
    ("Max Temp (°C)", '@max'),
    ('Station Name', '@sta')
]
hover = HoverTool(
    tooltips=tooltips,
    formatters={
        '@dt': 'datetime',
    })

hv.Scatter(
    hot_june[["dt", "max", "sta"]], vdims=["max", "sta"]
).opts(
    tools=[hover],
    size=5,
    width=600
)